In [ ]:
!pip install jax jaxlib
!pip install --quiet --upgrade scipy
!pip install --quiet jax jaxlib optax

In [ ]:
import jax
from jax.scipy.stats import norm
import jax.numpy as jnp
from scipy.stats import norm
import time
import jax.numpy as jnp
from numpy.polynomial.legendre import leggauss
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize_scalar, brentq, minimize
from scipy.special import gamma
from numpy.polynomial.legendre import leggauss
import warnings
warnings.filterwarnings('ignore')
from numpy.random import default_rng
from math import log
from numpy.random import default_rng, SeedSequence
from scipy.stats import kstwobign, cramervonmises, uniform
from joblib import Parallel, delayed
from itertools import zip_longest
from collections import OrderedDict


# ==============================================================
# Risk Measure Same J
# ==============================================================


In [ ]:
# --------------------------------------------
# 1) Load data and build the 4 samples
# --------------------------------------------
path = "/content/sample_data/norwegianfire_raw.csv"
df = pd.read_csv(path)
df["year_full"] = 1900 + df["year"].astype(int)

# Cap for “monster” claim
MONSTER_NOK = 2_000_000_000  # 2 billion NOK
MONSTER_kNOK = MONSTER_NOK / 1_000.0

# Strict threshold: keep claims EXCEEDING 500 kNOK
theta_kNOK = 500.0
df_work = df.loc[df["size"] > theta_kNOK].copy()
n_total = len(df)
n_work  = len(df_work)

def q_nearest(x: np.ndarray, p: float) -> float:
    """Robust 'nearest' quantile across pandas/numpy versions."""
    x = np.asarray(x, float)
    x = x[~np.isnan(x)]
    if x.size == 0:
        return np.nan
    s = pd.Series(x)
    try:
        return float(s.quantile(p, method="nearest"))
    except TypeError:
        try:
            return float(s.quantile(p, interpolation="nearest"))
        except TypeError:
            try:
                return float(np.quantile(x, p, method="nearest"))
            except TypeError:
                return float(np.percentile(x, 100*p, interpolation="nearest"))

# Headline stats (kNOK)
s = df_work["size"].astype(float)
n = s.size
m1 = s.mean()
q1 = q_nearest(s.values, 0.25)
q3 = q_nearest(s.values, 0.75)

m2 = np.mean((s - m1)**2)
m3 = np.mean((s - m1)**3)
g1 = m3 / (m2**1.5) if m2 > 0 else np.nan
skew_unbiased = (np.sqrt(n*(n-1))/(n-2))*g1 if n > 2 else np.nan

print("=== Norwegian fire insurance (kNOK), strict > 500 ===")
print(f"n_total: {n_total}")
print(f"n_after_theta (x>500): {n_work}")
print(f"year_min_max: ({int(df['year_full'].min())}, {int(df['year_full'].max())})")
print(f"min_kNOK: {s.min():,.0f}")
print(f"max_kNOK: {s.max():,.0f}")
print(f"mean_kNOK: {m1:,.3f}")
print(f"q1_kNOK: {q1:,.0f}")
print(f"q3_kNOK: {q3:,.0f}")
print(f"skewness: {skew_unbiased:.2f}")

# Original data (x > 500)
x_original = np.asarray(df_work["size"].values, dtype=float)
x_original = np.sort(x_original)

# Modified original: replace max by MONSTER_kNOK
x_mod_original = x_original.copy()
imax = np.argmax(x_mod_original)
x_mod_original[imax] = MONSTER_kNOK
x_mod_original = np.sort(x_mod_original)

# Sampled: n=50, seed=123
rng = np.random.default_rng(123)
idx50 = rng.choice(x_original.shape[0], size=50, replace=False)
x_sampled = np.sort(x_original[idx50])

# Modified sampled: replace its max with MONSTER_kNOK
x_mod_sampled = x_sampled.copy()
jmax = np.argmax(x_mod_sampled)
x_mod_sampled[jmax] = MONSTER_kNOK
x_mod_sampled = np.sort(x_mod_sampled)

# θ for ETELN code
θ = theta_kNOK

print(f"\nCounts: {len(x_original)}, {len(x_mod_original)}, {len(x_sampled)}, {len(x_mod_sampled)}")
print(f"Original max -> Modified original max: {np.max(x_original):,.0f} -> {np.max(x_mod_original):,.0f}")
print(f"Sampled max  -> Modified sampled max : {np.max(x_sampled):,.0f}  -> {np.max(x_mod_sampled):,.0f}")
print(f"θ (fixed): {θ:,.0f} kNOK")
print(f"Monster (cap): {MONSTER_kNOK:,.0f} kNOK  = {int(MONSTER_NOK):,} NOK")

def print_sample_block(x, title, rows=5, cols=10, integers=True):
    x = np.sort(np.asarray(x, float))
    if x.size != rows*cols:
        raise ValueError(f"Expected {rows*cols} values, got {x.size}.")
    if integers:
        x_disp = np.round(x).astype(int)
        fmt = lambda v: f"{v:,.0f}"
    else:
        x_disp = x
        fmt = lambda v: f"{v:,.3f}"
    print(title)
    for r in range(rows):
        start = r*cols
        row_vals = [fmt(v) for v in x_disp[start:start+cols]]
        print(", ".join(row_vals))
    print()

print_sample_block(x_sampled,      "Sampled data (n=50, kNOK):")
print_sample_block(x_mod_sampled,  "Modified sampled data (n=50, kNOK):")


=== Norwegian fire insurance (kNOK), strict > 500 ===
n_total: 9181
n_after_theta (x>500): 9020
year_min_max: (1972, 1992)
min_kNOK: 501
max_kNOK: 465,365
mean_kNOK: 2,247.860
q1_kNOK: 711
q3_kNOK: 1,817
skewness: 30.61

Counts: 9020, 9020, 50, 50
Original max -> Modified original max: 465,365 -> 2,000,000
Sampled max  -> Modified sampled max : 4,607  -> 2,000,000
θ (fixed): 500 kNOK
Monster (cap): 2,000,000 kNOK  = 2,000,000,000 NOK
Sampled data (n=50, kNOK):
505, 512, 513, 520, 541, 615, 632, 641, 650, 650
675, 675, 682, 698, 699, 700, 704, 704, 718, 735
745, 805, 822, 901, 911, 942, 957, 961, 1,035, 1,060
1,075, 1,223, 1,244, 1,343, 1,500, 1,743, 1,750, 2,039, 2,072, 2,097
2,139, 2,220, 2,294, 2,348, 2,468, 2,764, 3,317, 3,814, 4,377, 4,607

Modified sampled data (n=50, kNOK):
505, 512, 513, 520, 541, 615, 632, 641, 650, 650
675, 675, 682, 698, 699, 700, 704, 704, 718, 735
745, 805, 822, 901, 911, 942, 957, 961, 1,035, 1,060
1,075, 1,223, 1,244, 1,343, 1,500, 1,743, 1,750, 2,039, 2,

In [ ]:
# --------------------------------------------
# Performance knobs
# --------------------------------------------
try:
    from threadpoolctl import threadpool_limits
    _HAS_TPCTL = True
except Exception:
    _HAS_TPCTL = False

N_JOBS       = -1
BACKEND      = "loky"
BLAS_THREADS = 1

NQUAD_OBS = 800
NQUAD_MC  = 200
GRID_OBS  = 401
GRID_MC   = 201
MC_BATCH  = 200

MC_B_ORIG      = 10000
MC_B_MOD_ORIG  = 10000
MC_B_SAMP      = 2000
MC_B_MOD_SAMP  = 2000

ln2 = np.log(2.0)

# --------------------------------------------
# Generic GoF helpers & printing
# --------------------------------------------
def ks_statistic(x, cdf):
    x = np.sort(np.asarray(x, float)); n = x.size
    u = cdf(x); ecdf = (np.arange(1, n+1)) / n
    return float(np.max(np.abs(u - ecdf)))

def cvm_statistic(x, cdf):
    x = np.sort(np.asarray(x, float)); n = x.size
    u = cdf(x); i = np.arange(1, n+1)
    return float(np.sum((u - (2*i - 1)/(2*n))**2) + 1.0/(12*n))

def _panel_to_lines_with_spanner(title, rows):
    w = dict(est=20, a=6, b=7, pks=10, dks=8, pcvm=10, wcvm=9)

    def pad(s, width, align="<"):
        return f"{s:{align}{width}}"

    def fmt_p(p, det):
        return f"<{det:.3f}" if (det > 0 and p <= det) else f"{p:.3f}"

    det = rows[0].get("_det", 0.0) if rows else 0.0

    lines = [title]

    top = (
        pad("Estimator", w["est"]) +
        pad("α̂",        w["a"],  ">") +
        pad("β̂",        w["b"],  ">") +
        pad("KS Test",   w["pks"] + w["dks"], ">") +
        pad("CvM",       w["pcvm"] + w["wcvm"], ">")
    )
    sub = (
        pad("",          w["est"]) +
        pad("",          w["a"]) +
        pad("",          w["b"]) +
        pad("p",         w["pks"], ">") +
        pad("D",         w["dks"], ">") +
        pad("p",         w["pcvm"], ">") +
        pad("W",         w["wcvm"], ">")
    )
    lines += [top, sub]

    for r in rows:
        lines.append(
            pad(r["Estimator"],         w["est"]) +
            pad(f"{r['alpha']:.2f}",    w["a"],  ">") +
            pad(f"{r['beta']:.2f}",     w["b"],  ">") +
            pad(fmt_p(r["KS_p"], det),  w["pks"], ">") +
            pad(f"{r['KS_D']:.3f}",     w["dks"], ">") +
            pad(fmt_p(r["CvM_p"], det), w["pcvm"], ">") +
            pad(f"{r['CvM_W']:.3f}",    w["wcvm"], ">")
        )
    return lines

def print_side_by_side_with_spanners(left_title, left_rows, right_title, right_rows):
    L = _panel_to_lines_with_spanner(left_title, left_rows)
    R = _panel_to_lines_with_spanner(right_title, right_rows)
    gap = 3
    left_width = max(len(s) for s in L) + gap
    total_width = left_width + max(len(s) for s in R)
    rule = "=" * total_width
    print("\n" + rule)
    for l, r in zip_longest(L, R, fillvalue=""):
        print(f"{l:<{left_width}}{r}")
    print(rule)

# --------------------------------------------
# ETELN: CDF, sampling, MLE
# --------------------------------------------
def eteln_cdf(x, alpha, beta, theta):
    """
    CDF of ETELN(α, β, θ) with θ fixed; consistent with the simulation transform:
      if β ≈ 0:  F(x) = 1 + log Φ(α log(x/θ)) / log 2
      else:      F(x) = [2^β - Φ(α log(x/θ))^{-β}] / (2^β - 1)
    """
    x = np.asarray(x, float)
    u = np.zeros_like(x, dtype=float)

    m = x > theta
    if not np.any(m):
        return u

    z = alpha * np.log(x[m] / theta)
    G = norm.cdf(z)
    G = np.clip(G, 1e-300, 1.0 - 1e-15)

    if abs(beta) < 1e-8:
        u[m] = 1.0 + np.log(G) / ln2
    else:
        two_b = np.exp(beta * ln2)
        denom = two_b - 1.0
        if abs(denom) < 1e-14:
            denom = beta * ln2 + 0.5 * (beta**2) * (ln2**2)
        u[m] = (two_b - np.power(G, -beta)) / denom

    return np.clip(u, 0.0, 1.0)

def eteln_sample(n, alpha, beta, theta, rng=None):
    """
    Inverse-transform sampling for ETELN, matching your simulation code.
    """
    rng = default_rng() if rng is None else rng
    u = rng.uniform(0.0, 1.0, int(n))

    if abs(beta) < 1e-8:
        u0 = np.exp(-(1.0 - u) * ln2)   # 2^{-(1-u)}
    else:
        two_b = np.exp(beta * ln2)
        base  = two_b - (two_b - 1.0) * u
        base  = np.maximum(base, 1e-300)
        u0    = np.power(base, -1.0 / beta)

    u0 = np.clip(u0, 1e-9, 1.0 - 1e-9)
    xi = norm.ppf(u0)
    return theta * np.exp(xi / alpha)

def eteln_loglik_alpha_beta(x, alpha, beta, theta):
    """
    Log-likelihood for ETELN(α, β, θ) with θ fixed
    (mirrors your mle_eteln in the simulation class).
    """
    if alpha <= 0 or np.abs(beta) > 12:
        return -np.inf

    xv = np.asarray(x, float)
    xv = xv[xv > theta]
    if xv.size == 0:
        return -np.inf

    n = xv.size

    if abs(beta) < 1e-8:
        const = -np.log(np.log(2.0))
    else:
        two_b = np.exp(beta * ln2)
        const = np.log(np.abs(beta)) - np.log(np.abs(two_b - 1.0))

    z = alpha * np.log(xv / theta)
    phi_z = norm.pdf(z)
    Phi_z = norm.cdf(z)

    Phi_z = np.maximum(Phi_z, 1e-300)
    phi_z = np.maximum(phi_z, 1e-300)

    ll = (
        n * np.log(alpha) + n * const
        - 0.5 * n * np.log(2.0 * np.pi)
        - np.sum(np.log(xv))
        - 0.5 * alpha**2 * np.sum(np.log(xv / theta)**2)
        - (beta + 1.0) * np.sum(np.log(Phi_z))
    )

    return float(ll) if np.isfinite(ll) else -np.inf

def fit_mle_eteln(x, theta):
    x = np.asarray(x, float)
    x = x[x > theta]
    if x.size < 5:
        return np.nan, np.nan

    lx = np.log(x)
    m1 = lx.mean()
    m2 = (lx**2).mean()
    var = max(m2 - m1**2, 1e-6)
    a0 = max(0.1, 1.0 / np.sqrt(var))
    b0 = 0.5

    def nll(p):
        a, b = p
        v = eteln_loglik_alpha_beta(x, a, b, theta)
        return -v if np.isfinite(v) else 1e20

    res = minimize(
        nll,
        x0=[a0, b0],
        bounds=[(1e-3, 40.0), (-8.0, 8.0)],
        method="L-BFGS-B"
    )
    if res.success:
        return float(res.x[0]), float(res.x[1])

    res = minimize(
        nll,
        x0=[1.5, 0.2],
        bounds=[(1e-3, 40.0), (-8.0, 8.0)],
        method="L-BFGS-B"
    )
    return (float(res.x[0]), float(res.x[1])) if res.success else (np.nan, np.nan)

# --------------------------------------------
# Same-J L-estimation for ETELN
# --------------------------------------------
def _dk_tau(beta, a, b, n_quad):
    """
    For ETELN same-J:
      d1(β) = ∫ J(u;a,b) ξ(u;β) du
      d2(β) = ∫ J(u;a,b) ξ(u;β)^2 du
      τ(β)  = d2(β) - d1(β)^2
    with ξ(u;β) from the ETELN ETE transform.
    """
    nodes, w = np.polynomial.legendre.leggauss(n_quad)
    u = 0.5 * (nodes + 1.0)
    w = 0.5 * w

    J = a * b * (u**(a - 1.0)) * ((1.0 - u**a)**(b - 1.0))

    if abs(beta) < 1e-10:
        u0 = np.exp(-(1.0 - u) * ln2)
    else:
        two_b = np.exp(beta * ln2)
        base  = two_b - (two_b - 1.0) * u
        base  = np.maximum(base, 1e-300)
        u0    = np.power(base, -1.0 / beta)

    u0 = np.clip(u0, 1e-9, 1.0 - 1e-9)
    xi = norm.ppf(u0)

    d1 = float(np.sum(w * J * xi))
    d2 = float(np.sum(w * J * (xi**2)))
    tau = d2 - d1**2
    return d1, max(tau, 1e-14)

def fit_L_eteln_stable(x, a=1.1, b=1.2, theta=500.0,
                       n_quad=NQUAD_OBS, root_grid=GRID_OBS):
    """
    Same-J L-estimator for ETELN:
      (μ1 - log θ)/sqrt(Δ) = d1(β)/sqrt(τ(β)),
    then α̂ = sqrt(τ̂/Δ).
    """
    x = np.asarray(x, float)
    x = x[x > theta]
    n = x.size
    if n < 5:
        return np.nan, np.nan

    xs = np.sort(x)
    i = np.arange(1, n + 1)
    uo = i / (n + 1.0)

    Jw = a * b * (uo**(a - 1.0)) * ((1.0 - uo**a)**(b - 1.0))
    wts = Jw / np.sum(Jw)

    lx = np.log(xs)
    mu1 = float(np.sum(wts * lx))
    mu2 = float(np.sum(wts * (lx**2)))
    Delta = max(mu2 - mu1**2, 1e-12)

    target = (mu1 - np.log(theta)) / np.sqrt(Delta)

    def R(beta):
        d1, tau = _dk_tau(beta, a, b, n_quad=n_quad)
        return d1 / np.sqrt(tau) - target

    grid = np.linspace(-10.0, 10.0, int(root_grid))
    vals = np.array([R(bi) for bi in grid])
    sgn = np.sign(vals)

    beta_hat = None
    for k in range(len(grid) - 1):
        if (np.isfinite(vals[k]) and np.isfinite(vals[k + 1]) and
                sgn[k] * sgn[k + 1] < 0):
            beta_hat = brentq(R, grid[k], grid[k + 1],
                              xtol=1e-8, maxiter=400)
            break

    if beta_hat is None:
        def obj(b):
            rb = R(b)
            return (rb if np.isfinite(rb) else 1e6)**2
        res = minimize_scalar(
            obj,
            bounds=(-10.0, 10.0),
            method="bounded",
            options={"xatol": 1e-8, "maxiter": 1000}
        )
        beta_hat = float(res.x)

    _, tau_hat = _dk_tau(beta_hat, a, b, n_quad=n_quad)
    alpha_hat = np.sqrt(tau_hat / Delta)
    return float(alpha_hat), float(beta_hat)

def make_safe_L_factory_eteln(a_req, b_req, fallbacks=None,
                              n_quad=NQUAD_OBS, root_grid=GRID_OBS):
    if fallbacks is None:
        fallbacks = [(1.05, 1.10), (1.10, 1.20), (1.20, 1.30)]

    def fit_fun(z, _nq=n_quad, _rg=root_grid):
        ah, bh = fit_L_eteln_stable(z, a_req, b_req, θ,
                                    n_quad=_nq, root_grid=_rg)
        if np.isfinite(ah) and np.isfinite(bh):
            return ah, bh

        for (aa, bb) in fallbacks:
            ah2, bh2 = fit_L_eteln_stable(z, aa, bb, θ,
                                          n_quad=_nq, root_grid=_rg)
            if np.isfinite(ah2) and np.isfinite(bh2):
                return ah2, bh2

        return fit_mle_eteln(z, θ)

    return fit_fun

# --------------------------------------------
# Monte Carlo p-values for ETELN
# --------------------------------------------
def mc_pvals_parallel_eteln(x, fit_fun, theta, B=2000, seed=1234,
                            n_jobs=N_JOBS, backend=BACKEND,
                            nquad_obs=NQUAD_OBS, nquad_mc=NQUAD_MC,
                            grid_obs=GRID_OBS, grid_mc=GRID_MC,
                            batch=MC_BATCH):
    if _HAS_TPCTL:
        threadpool_limits(BLAS_THREADS)

    x = np.asarray(x, float)
    x = x[x > theta]
    n = x.size
    if n == 0:
        return (np.nan, np.nan, np.nan, np.nan, np.nan, np.nan)

    def fit_obs(z):
        return fit_fun(z, _nq=nquad_obs, _rg=grid_obs)

    a_hat, b_hat = fit_obs(x)
    cdf_hat = lambda z: eteln_cdf(z, a_hat, b_hat, theta)
    D_obs = ks_statistic(x, cdf_hat)
    W_obs = cvm_statistic(x, cdf_hat)

    seeds = SeedSequence(seed).spawn(B)
    seed_ints = [int(s.generate_state(1)[0]) for s in seeds]

    def one_rep(seed_i):
        rng = default_rng(seed_i)
        xb = eteln_sample(n, a_hat, b_hat, theta, rng)
        ah, bh = fit_fun(xb, _nq=nquad_mc, _rg=grid_mc)
        cdf_b = lambda z, aa=ah, bb=bh: eteln_cdf(z, aa, bb, theta)
        return ks_statistic(xb, cdf_b), cvm_statistic(xb, cdf_b)

    def _chunks(lst, k):
        for i in range(0, len(lst), k):
            yield lst[i:i+k]

    Ds_all, Ws_all = [], []
    for chunk in _chunks(seed_ints, batch):
        Ds, Ws = zip(*Parallel(n_jobs=n_jobs, backend=backend)(
            delayed(one_rep)(si) for si in chunk
        ))
        Ds_all.append(np.array(Ds))
        Ws_all.append(np.array(Ws))

    Ds = np.concatenate(Ds_all)
    Ws = np.concatenate(Ws_all)

    p_ks  = (1.0 + np.sum(Ds >= D_obs)) / (B + 1.0)
    p_cvm = (1.0 + np.sum(Ws >= W_obs)) / (B + 1.0)
    return (a_hat, b_hat, D_obs, W_obs, p_ks, p_cvm)

def mc_pvals_fixedparams_eteln(x, fit_fun, theta, B=10000, seed=1234,
                               n_jobs=N_JOBS, backend=BACKEND,
                               nquad_obs=NQUAD_OBS, batch=MC_BATCH):
    if _HAS_TPCTL:
        threadpool_limits(BLAS_THREADS)

    x = np.asarray(x, float)
    x = x[x > theta]
    n = x.size
    if n == 0:
        return (np.nan, np.nan, np.nan, np.nan, np.nan, np.nan)

    a_hat, b_hat = fit_fun(x, _nq=nquad_obs, _rg=GRID_OBS)
    cdf_hat = lambda z: eteln_cdf(z, a_hat, b_hat, theta)
    D_obs = ks_statistic(x, cdf_hat)
    W_obs = cvm_statistic(x, cdf_hat)

    seeds = SeedSequence(seed).spawn(B)
    seed_ints = [int(s.generate_state(1)[0]) for s in seeds]

    def one_rep(seed_i):
        rng = default_rng(seed_i)
        xb = eteln_sample(n, a_hat, b_hat, theta, rng)
        return ks_statistic(xb, cdf_hat), cvm_statistic(xb, cdf_hat)

    def _chunks(lst, k):
        for i in range(0, len(lst), k):
            yield lst[i:i+k]

    Ds_all, Ws_all = [], []
    for chunk in _chunks(seed_ints, batch):
        Ds, Ws = zip(*Parallel(n_jobs=n_jobs, backend=backend)(
            delayed(one_rep)(si) for si in chunk
        ))
        Ds_all.append(np.array(Ds))
        Ws_all.append(np.array(Ws))

    Ds = np.concatenate(Ds_all)
    Ws = np.concatenate(Ws_all)

    p_ks  = (1.0 + np.sum(Ds >= D_obs)) / (B + 1.0)
    p_cvm = (1.0 + np.sum(Ws >= W_obs)) / (B + 1.0)
    return (a_hat, b_hat, D_obs, W_obs, p_ks, p_cvm)

# --------------------------------------------
# GoF wrapper & panel builder for ETELN
# --------------------------------------------
def gof_with_mode_eteln(x, fit_fun, mode="mc", B=100, seed=1234, n_jobs=N_JOBS):
    x = np.asarray(x, float)
    x = x[x > θ]
    if x.size == 0:
        return {
            "Estimator": "", "alpha": np.nan, "beta": np.nan,
            "KS_p": np.nan, "KS_D": np.nan,
            "CvM_p": np.nan, "CvM_W": np.nan,
            "_det": 1.0 / (B + 1.0)
        }

    if mode == "mc_fixed":
        a_hat, b_hat, D_obs, W_obs, p_ks, p_cvm = mc_pvals_fixedparams_eteln(
            x, fit_fun, θ, B=B, seed=seed, n_jobs=n_jobs,
            nquad_obs=NQUAD_OBS, batch=MC_BATCH
        )
        return {
            "Estimator": "", "alpha": a_hat, "beta": b_hat,
            "KS_p": p_ks, "KS_D": D_obs,
            "CvM_p": p_cvm, "CvM_W": W_obs,
            "_det": 1.0 / (B + 1.0)
        }

    if mode == "mc":
        a_hat, b_hat, D_obs, W_obs, p_ks, p_cvm = mc_pvals_parallel_eteln(
            x, fit_fun, θ, B=B, seed=seed, n_jobs=n_jobs,
            nquad_obs=NQUAD_OBS, nquad_mc=NQUAD_MC,
            grid_obs=GRID_OBS, grid_mc=GRID_MC
        )
        return {
            "Estimator": "", "alpha": a_hat, "beta": b_hat,
            "KS_p": p_ks, "KS_D": D_obs,
            "CvM_p": p_cvm, "CvM_W": W_obs,
            "_det": 1.0 / (B + 1.0)
        }

    # asymptotic fallback (not used in your MC design)
    a_hat, b_hat = fit_fun(x)
    cdf_hat = lambda z: eteln_cdf(z, a_hat, b_hat, θ)
    D_obs = ks_statistic(x, cdf_hat)
    W_obs = cvm_statistic(x, cdf_hat)
    p_ks = float(kstwobign.sf(np.sqrt(x.size) * D_obs))
    u = eteln_cdf(np.sort(x), a_hat, b_hat, θ)
    p_cvm = float(cramervonmises(u, uniform.cdf).pvalue)
    return {
        "Estimator": "", "alpha": a_hat, "beta": b_hat,
        "KS_p": p_ks, "KS_D": D_obs,
        "CvM_p": p_cvm, "CvM_W": W_obs,
        "_det": 0.0
    }

def build_panel_eteln(x, estimators, mode="mc", B=100, seed=1234, n_jobs=N_JOBS):
    rows = []
    for (name, fit_fun) in estimators:
        r = gof_with_mode_eteln(x, fit_fun, mode=mode, B=B, seed=seed, n_jobs=n_jobs)
        r["Estimator"] = name
        rows.append(r)
    return rows


In [ ]:
# ==============================================================================
# ETELN RISK MEASURES (Same-J, Norwegian Fire Insurance)
# ==============================================================================

import numpy as np
from numpy.polynomial.legendre import leggauss
from scipy.stats import norm

# Uses the same global theta as the rest of your ETELN code
# θ = 500.0   # already defined earlier

# Any CTE larger than this will be treated as ∞ (just for presentation)
CTE_GUARD_ETELN = 2_000_000.0

# ------------------------------------------------------------------------------
# ETELN quantile, VaR and CTE
# ------------------------------------------------------------------------------

class ETELNRiskMeasures:
    def __init__(self, theta=500.0, n_quad=400):
        self.theta = float(theta)
        self.n_quad = int(n_quad)

        nodes, weights = leggauss(self.n_quad)
        self.u = 0.5 * (nodes + 1.0)   # map [-1,1] -> [0,1]
        self.w = 0.5 * weights

        self.ln2 = np.log(2.0)

    def eteln_quantile(self, u, alpha, beta):
        """
        ETELN quantile, consistent with your ETELN sampler:
            X = θ * exp( ξ / α ),  where ξ = Φ^{-1}(U0(β))
        """
        u = np.clip(np.asarray(u, float), 1e-12, 1.0 - 1e-12)

        if abs(beta) < 1e-10:
            u0 = np.exp(-(1.0 - u) * self.ln2)
        else:
            two_b = np.exp(beta * self.ln2)
            base  = two_b - (two_b - 1.0) * u
            base  = np.clip(base, 1e-300, None)
            u0    = base**(-1.0 / beta)

        u0 = np.clip(u0, 1e-12, 1.0 - 1e-12)
        xi  = norm.ppf(u0)
        return self.theta * np.exp(xi / alpha)

    def var(self, alpha, beta, confidence_level=0.99):
        return float(self.eteln_quantile(confidence_level, alpha, beta))

    def cte(self, alpha, beta, confidence_level=0.99):
        """
        CTE for ETELN. All moments are finite for α>0, so no α-threshold like ETELL.
        We apply only a *numerical* guard later when building tables.
        """
        p = float(confidence_level)
        s = p + (1.0 - p) * self.u           # s ∈ [p,1]
        q_vals = self.eteln_quantile(s, alpha, beta)
        return float(np.sum(self.w * q_vals))

    def compute_risk_measures(self, alpha, beta,
                              confidence_levels=(0.98, 0.99, 0.995)):
        out = {}
        for cl in confidence_levels:
            var_val = self.var(alpha, beta, cl)
            cte_val = self.cte(alpha, beta, cl)

            # Large-value guard: present huge CTEs as ∞
            if np.isfinite(cte_val) and cte_val > CTE_GUARD_ETELN:
                cte_val = float('inf')

            out[cl] = {
                "confidence_level": cl,
                "tail_probability": 1.0 - cl,
                "VaR": var_val,
                "CTE": cte_val,
            }
        return out


# ------------------------------------------------------------------------------
# Empirical risk measures
# ------------------------------------------------------------------------------

def empirical_var(x, confidence_level=0.99):
    x = np.sort(np.asarray(x, float))
    return float(np.quantile(x, confidence_level, method="linear"))

def empirical_cte(x, confidence_level=0.99):
    x = np.asarray(x, float)
    v = empirical_var(x, confidence_level)
    exc = x[x > v]
    if exc.size == 0:
        return np.nan
    return float(exc.mean())


# ------------------------------------------------------------------------------
# Build ETELN risk table using SAME J-weights as ETELL Same-J block
# ------------------------------------------------------------------------------

def compute_risk_table_eteln(data_dict, estimators,
                             confidence_levels=(0.98, 0.99, 0.995),
                             theta=500.0, n_quad=400):
    rm = ETELNRiskMeasures(theta=theta, n_quad=n_quad)
    results = {}

    for data_name, x in data_dict.items():
        x = np.asarray(x, float)
        x = x[x > theta]
        if x.size == 0:
            continue

        results[data_name] = {}

        # ----- empirical -----
        emp_dict = {}
        for cl in confidence_levels:
            v_emp = empirical_var(x, cl)
            c_emp = empirical_cte(x, cl)
            if np.isfinite(c_emp) and c_emp > CTE_GUARD_ETELN:
                c_emp = float('inf')
            emp_dict[cl] = {"VaR": v_emp, "CTE": c_emp}
        results[data_name]["Empirical"] = emp_dict

        # ----- fitted models -----
        for est_name, fit_fun in estimators:
            try:
                alpha_hat, beta_hat = fit_fun(x)
            except Exception as e:
                print(f"Warning: {est_name} failed on {data_name}: {e}")
                results[data_name][est_name] = None
                continue

            if not (np.isfinite(alpha_hat) and np.isfinite(beta_hat)):
                results[data_name][est_name] = None
                continue

            model_res = rm.compute_risk_measures(alpha_hat, beta_hat,
                                                 confidence_levels)
            model_res["alpha"] = alpha_hat
            model_res["beta"]  = beta_hat
            results[data_name][est_name] = model_res

    return results


# ------------------------------------------------------------------------------
# Pretty-print helpers (same style as ETELL block)
# ------------------------------------------------------------------------------

_INF_THRESHOLD = 1e15  # anything larger is printed as ∞

def _fmt(v):
    if v is None or not np.isfinite(v) or v > _INF_THRESHOLD:
        return "∞"
    return f"{v:,.2f}"

def _fmtdiff(a, b):
    bad = lambda z: (z is None) or (not np.isfinite(z)) or (z > _INF_THRESHOLD)
    if bad(a) or bad(b):
        return "—"
    return f"{(a - b):+,.2f}"


def print_risk_table_eteln(results, confidence_levels=(0.98, 0.99, 0.995)):
    for data_name, data_results in results.items():
        print("\n" + "="*100)
        print(f"ETELN Risk Measures: {data_name}")
        print("="*100)

        estimators = [k for k in data_results.keys() if k != "Empirical"]

        for cl in confidence_levels:
            tail_prob = 1.0 - cl
            print(f"\n{'Confidence Level':<20} {cl*100:.1f}% (Tail Probability = {tail_prob*100:.2f}%)")
            print("-"*100)

            # header
            print(f"{'Method':<20} {'α̂':>8} {'β̂':>8} "
                  f"{'VaR':>15} {'CTE':>15} {'VaR Diff':>15} {'CTE Diff':>15}")
            print("-"*100)

            # empirical row
            emp = data_results["Empirical"][cl]
            print(f"{'Empirical':<20} {'-':>8} {'-':>8} "
                  f"{_fmt(emp['VaR']):>15} {_fmt(emp['CTE']):>15} "
                  f"{'-':>15} {'-':>15}")

            # model rows
            for est_name in estimators:
                est_results = data_results[est_name]
                if est_results is None or cl not in est_results:
                    print(f"{est_name:<20} {'FAILED':>8} {'FAILED':>8} "
                          f"{'-':>15} {'-':>15} {'-':>15} {'-':>15}")
                    continue

                alpha_hat = est_results["alpha"]
                beta_hat  = est_results["beta"]
                var_theo  = est_results[cl]["VaR"]
                cte_theo  = est_results[cl]["CTE"]

                print(f"{est_name:<20} {alpha_hat:>8.2f} {beta_hat:>8.2f} "
                      f"{_fmt(var_theo):>15} {_fmt(cte_theo):>15} "
                      f"{_fmtdiff(var_theo, emp['VaR']):>15} "
                      f"{_fmtdiff(cte_theo, emp['CTE']):>15}")


def print_comparison_table_eteln(results, confidence_level=0.99):
    datasets   = list(results.keys())
    first_data = results[datasets[0]]
    estimators = [k for k in first_data.keys() if k != "Empirical"]

    print("\n" + "="*140)
    print(f"ETELN Risk Measures Comparison at {confidence_level*100:.1f}% Confidence Level")
    print("="*140)

    header = f"{'Method':<20}"
    for data_name in datasets:
        header += f" | {data_name:^28}"
    print(header)

    subheader = f"{'':<20}"
    for _ in datasets:
        subheader += f" | {'VaR':>13} {'CTE':>13}"
    print(subheader)
    print("-"*140)

    # empirical row
    row = f"{'Empirical':<20}"
    for data_name in datasets:
        emp = results[data_name]["Empirical"][confidence_level]
        row += f" | {_fmt(emp['VaR']):>13} {_fmt(emp['CTE']):>13}"
    print(row)
    print("-"*140)

    # model rows (MLE first)
    for est_name in ["MLE"] + [e for e in estimators if e != "MLE"]:
        row = f"{est_name:<20}"
        for data_name in datasets:
            est_results = results[data_name].get(est_name)
            if est_results is None or confidence_level not in est_results:
                row += f" | {'FAILED':>13} {'FAILED':>13}"
            else:
                var_theo = est_results[confidence_level]["VaR"]
                cte_theo = est_results[confidence_level]["CTE"]
                row += f" | {_fmt(var_theo):>13} {_fmt(cte_theo):>13}"
        print(row)

    print("="*140)


# ------------------------------------------------------------------------------
# RUN BLOCK (Same J-weights as your ETELL Same-J risk table)
# ------------------------------------------------------------------------------

if __name__ == "__main__":

    data_dict_eteln = {
        "Sampled (n=50)":       x_sampled,
        "Modified Sampled":     x_mod_sampled,
        "Original (n=9020)":    x_original,
        "Modified Original":    x_mod_original,
    }

    # Same J-weights as ETELL Same-J block
    estimators_eteln = [
        ("MLE",         lambda z: fit_mle_eteln(z, θ)),
        ("J(1.25,9.00)", make_safe_L_factory_eteln(1.25, 9.00)),
        ("J(1.35,3.50)", make_safe_L_factory_eteln(1.35, 3.50)),
        ("J(1.30,1.60)", make_safe_L_factory_eteln(1.30, 1.60)),
        ("J(1.15,4.00)", make_safe_L_factory_eteln(1.15, 4.00)),
    ]

    print("Computing ETELN risk measures (Same-J)...")
    results_eteln = compute_risk_table_eteln(
        data_dict=data_dict_eteln,
        estimators=estimators_eteln,
        confidence_levels=(0.98, 0.99, 0.995),
        theta=θ,
        n_quad=400,
    )

    print_risk_table_eteln(results_eteln, confidence_levels=(0.98, 0.99, 0.995))
    print_comparison_table_eteln(results_eteln, confidence_level=0.98)
    print_comparison_table_eteln(results_eteln, confidence_level=0.99)
    print_comparison_table_eteln(results_eteln, confidence_level=0.995)


Computing ETELN risk measures (Same-J)...

ETELN Risk Measures: Sampled (n=50)

Confidence Level     98.0% (Tail Probability = 2.00%)
----------------------------------------------------------------------------------------------------
Method                     α̂       β̂             VaR             CTE        VaR Diff        CTE Diff
----------------------------------------------------------------------------------------------------
Empirical                   -        -        4,381.60        4,607.00               -               -
MLE                      0.95    -0.60        5,448.49        8,333.96       +1,066.89       +3,726.96
J(1.25,9.00)             0.99    -0.83        5,155.65        7,726.73         +774.05       +3,119.73
J(1.35,3.50)             0.83     0.06        7,005.99       11,725.00       +2,624.39       +7,118.00
J(1.30,1.60)             0.83     0.04        6,974.06       11,644.60       +2,592.46       +7,037.60
J(1.15,4.00)             0.95    -0.57        

# ==============================================================
# Risk Measure Different J
# ==============================================================


In [ ]:
# ==============================================================
# DESIGN B: Real Data Analysis - Norwegian Fire Insurance (ETELN)
# Different J (J₁ ≠ J₂), Same h = log(x)
# ==============================================================

# --------------------------------------------
# PERFORMANCE KNOBS
# --------------------------------------------
try:
    from threadpoolctl import threadpool_limits
    _HAS_TPCTL = True
except:
    _HAS_TPCTL = False

N_JOBS       = -1
BACKEND      = "loky"
BLAS_THREADS = 1

NQUAD_OBS = 800
NQUAD_MC  = 200
GRID_OBS  = 401
GRID_MC   = 201
MC_BATCH  = 200

MC_B_ORIG      = 10000
MC_B_MOD_ORIG  = 10000
MC_B_SAMP      = 2000
MC_B_MOD_SAMP  = 2000

ln2 = np.log(2.0)

# --------------------------------------------
# ETELN CDF
# --------------------------------------------
def eteln_cdf(x, alpha, beta, theta):
    x = np.asarray(x, float)
    u = np.zeros_like(x)
    m = x > theta
    if not np.any(m):
        return u

    z = alpha * np.log(x[m] / theta)
    G = np.clip(norm.cdf(z), 1e-300, 1 - 1e-15)

    if abs(beta) < 1e-10:
        u[m] = 1.0 + np.log(G) / ln2
    else:
        two_b = np.exp(beta * ln2)
        denom = two_b - 1
        u[m] = (two_b - G**(-beta)) / denom

    return np.clip(u, 0, 1)

# --------------------------------------------
# ETELN SAMPLING
# --------------------------------------------
def eteln_sample(n, alpha, beta, theta, rng=None):
    rng = default_rng() if rng is None else rng
    u = rng.uniform(0, 1, n)

    if abs(beta) < 1e-10:
        u0 = np.exp(-(1 - u) * ln2)
    else:
        two_b = np.exp(beta * ln2)
        base = two_b - (two_b - 1) * u
        u0 = np.clip(base, 1e-300, None)**(-1/beta)

    u0 = np.clip(u0, 1e-12, 1 - 1e-12)
    xi = norm.ppf(u0)
    return theta * np.exp(xi / alpha)

# --------------------------------------------
# ETELN LOG-LIKELIHOOD & MLE
# --------------------------------------------
def eteln_loglik_alpha_beta(x, alpha, beta, theta):
    if alpha <= 0 or abs(beta) > 12:
        return -np.inf
    xv = x[x > theta]
    if len(xv) == 0:
        return -np.inf
    n = len(xv)

    if abs(beta) < 1e-10:
        const = -np.log(np.log(2))
    else:
        two_b = np.exp(beta * ln2)
        const = np.log(abs(beta)) - np.log(abs(two_b - 1))

    z = alpha * np.log(xv/theta)
    phi_z = np.maximum(norm.pdf(z), 1e-300)
    Phi_z = np.maximum(norm.cdf(z), 1e-300)

    ll = (
        n*np.log(alpha) + n*const
        - 0.5*n*np.log(2*np.pi)
        - np.sum(np.log(xv))
        - 0.5*alpha**2 * np.sum(np.log(xv/theta)**2)
        - (beta + 1)*np.sum(np.log(Phi_z))
    )
    return float(ll)

def fit_mle_eteln(x, theta):
    x = np.asarray(x, float)
    x = x[x > theta]
    if len(x) < 5:
        return np.nan, np.nan

    lx = np.log(x)
    var = max(np.var(lx), 1e-6)
    a0 = 1.0 / np.sqrt(var)
    b0 = 0.5

    def nll(p):
        a, b = p
        return -eteln_loglik_alpha_beta(x, a, b, theta)

    res = minimize(nll, x0=[a0, b0], bounds=[(1e-3, 40), (-8, 8)], method="L-BFGS-B")
    if res.success:
        return res.x
    return np.nan, np.nan

# --------------------------------------------
# DESIGN B — CORE INTEGRALS
# --------------------------------------------
def _cw_designB_eteln(beta, a1, b1, a2, b2, n_quad):
    nodes, w = np.polynomial.legendre.leggauss(n_quad)
    u = 0.5*(nodes + 1)
    w = 0.5*w

    J1 = a1*b1*(u**(a1-1)) * (1 - u**a1)**(b1-1)
    J2 = a2*b2*(u**(a2-1)) * (1 - u**a2)**(b2-1)

    # ETELN latent ξ(u;β)
    if abs(beta) < 1e-10:
        u0 = np.exp(-(1 - u)*ln2)
    else:
        two_b = np.exp(beta*ln2)
        base = two_b - (two_b - 1)*u
        u0 = np.clip(base, 1e-300, None)**(-1/beta)

    u0 = np.clip(u0, 1e-12, 1 - 1e-12)
    xi = norm.ppf(u0)

    c1 = np.sum(w * J1 * xi)
    c2 = np.sum(w * J2 * xi)
    cw = c2 - c1
    return c1, c2, cw

# --------------------------------------------
# DESIGN B L-ESTIMATOR
# --------------------------------------------
def fit_L_eteln_designB_stable(x, a1, b1, a2, b2, theta=θ,
                               n_quad=NQUAD_OBS, root_grid=GRID_OBS):

    x = np.asarray(x, float)
    x = x[x > theta]
    n = len(x)
    if n < 5:
        return np.nan, np.nan

    xs = np.sort(x)
    uo = np.arange(1, n+1) / (n+1)

    J1 = a1*b1*(uo**(a1-1)) * (1 - uo**a1)**(b1-1)
    J2 = a2*b2*(uo**(a2-1)) * (1 - uo**a2)**(b2-1)

    J1 = J1 / np.sum(J1)
    J2 = J2 / np.sum(J2)

    lx = np.log(xs)
    mu1 = np.sum(J1 * lx)
    mu2 = np.sum(J2 * lx)
    Delta_w = mu2 - mu1
    if abs(Delta_w) < 1e-12:
        return np.nan, np.nan

    def Psi(beta):
        c1, c2, cw = _cw_designB_eteln(beta, a1, b1, a2, b2, n_quad)
        if abs(cw) < 1e-14:
            return np.nan
        return c1/cw + (np.log(theta) - mu1)/Delta_w

    grid = np.linspace(-10, 10, root_grid)
    vals = np.array([Psi(b) for b in grid])
    s = np.sign(vals)

    beta_hat = None
    for i in range(len(grid)-1):
        if np.isfinite(vals[i]) and np.isfinite(vals[i+1]) and s[i]*s[i+1] < 0:
            try:
                beta_hat = brentq(Psi, grid[i], grid[i+1], xtol=1e-8, maxiter=500)
                break
            except:
                pass

    if beta_hat is None:
        obj = lambda b: (Psi(b) if np.isfinite(Psi(b)) else 1e6)**2
        beta_hat = minimize_scalar(obj, bounds=(-10, 10),
                                   method="bounded",
                                   options={'xatol':1e-8}).x

    _, _, cw_hat = _cw_designB_eteln(beta_hat, a1, b1, a2, b2, n_quad)
    alpha_hat = cw_hat / Delta_w
    if alpha_hat <= 0:
        return np.nan, np.nan

    return float(alpha_hat), float(beta_hat)

# --------------------------------------------
# FACTORY: CREATE A SAFE L-FUNCTION
# --------------------------------------------
def make_safe_L_designB_factory_eteln(a1, b1, a2, b2,
                                      fallbacks=None,
                                      n_quad=NQUAD_OBS, root_grid=GRID_OBS):

    if fallbacks is None:
        fallbacks = [
            ((1.0,1.0),(1.0,2.0)),
            ((1.0,1.0),(2.0,1.0)),
            ((1.0,1.0),(1.5,1.5)),
        ]

    def fit_fun(z, _nq=n_quad, _rg=root_grid):
        ah, bh = fit_L_eteln_designB_stable(z, a1, b1, a2, b2,
                                            theta=θ,
                                            n_quad=_nq, root_grid=_rg)
        if np.isfinite(ah) and np.isfinite(bh):
            return ah, bh

        for (p1, p2) in fallbacks:
            ah2, bh2 = fit_L_eteln_designB_stable(z, p1[0], p1[1], p2[0], p2[1],
                                                  theta=θ,
                                                  n_quad=_nq, root_grid=_rg)
            if np.isfinite(ah2) and np.isfinite(bh2):
                return ah2, bh2

        return fit_mle_eteln(z, θ)

    return fit_fun

# --------------------------------------------
# MC functions (parameter MC + refit MC)
# --------------------------------------------
def mc_pvals_fixedparams_designB_eteln(x, fit_fun, theta, B=10000, seed=1234):
    if _HAS_TPCTL: threadpool_limits(BLAS_THREADS)

    x = np.asarray(x, float)
    x = x[x > theta]
    n = len(x)

    a_hat, b_hat = fit_fun(x, _nq=NQUAD_OBS, _rg=GRID_OBS)

    cdf = lambda z: eteln_cdf(z, a_hat, b_hat, theta)
    D_obs = ks_statistic(x, cdf)
    W_obs = cvm_statistic(x, cdf)

    seeds = SeedSequence(seed).spawn(B)
    Ds = []
    Ws = []
    for s in seeds:
        bx = eteln_sample(n, a_hat, b_hat, theta, rng=default_rng(int(s.generate_state(1)[0])))
        Ds.append(ks_statistic(bx, cdf))
        Ws.append(cvm_statistic(bx, cdf))

    Ds = np.array(Ds)
    Ws = np.array(Ws)

    pks = (1 + np.sum(Ds >= D_obs)) / (B + 1)
    pcv = (1 + np.sum(Ws >= W_obs)) / (B + 1)

    return a_hat, b_hat, D_obs, W_obs, pks, pcv

# --------------------------------------------
# GOF WRAPPER
# --------------------------------------------
def gof_with_mode_designB_eteln(x, fit_fun, mode="mc_fixed", B=10000, seed=2025):
    x = np.asarray(x, float); x = x[x > θ]
    if len(x) == 0:
        return {
            "Estimator":"","alpha":np.nan,"beta":np.nan,
            "KS_p":np.nan,"KS_D":np.nan,
            "CvM_p":np.nan,"CvM_W":np.nan,
            "_det":1/(B+1)
        }

    if mode == "mc_fixed":
        a,b,D,W,pks,pcv = mc_pvals_fixedparams_designB_eteln(
            x, fit_fun, θ, B=B, seed=seed
        )
        return {"Estimator":"", "alpha":a,"beta":b,
                "KS_p":pks,"KS_D":D,
                "CvM_p":pcv,"CvM_W":W,
                "_det":1/(B+1)}

    # asymptotic fallback
    a,b = fit_fun(x)
    cdf = lambda z: eteln_cdf(z,a,b,θ)
    D = ks_statistic(x,cdf)
    W = cvm_statistic(x,cdf)
    p_ks = float(kstwobign.sf(np.sqrt(len(x))*D))
    p_cv = float(cramervonmises(eteln_cdf(np.sort(x),a,b,θ), uniform.cdf).pvalue)
    return {"Estimator":"", "alpha":a,"beta":b,
            "KS_p":p_ks,"KS_D":D,
            "CvM_p":p_cv,"CvM_W":W,
            "_det":0.0}

# --------------------------------------------
# PANEL BUILDER + PRINTING
# --------------------------------------------
def build_panel_designB_eteln(x, estimators, mode="mc_fixed", B=10000, seed=2025):
    rows=[]
    for (name,fit_fun) in estimators:
        r = gof_with_mode_designB_eteln(x,fit_fun,mode,B,seed)
        r["Estimator"]=name
        rows.append(r)
    return rows

def print_side_by_side_with_spanners(left_title, left_rows,
                                     right_title, right_rows):
    # same printer as ETLL
    w = dict(est=20, a=6, b=7, pks=10, dks=8, pcvm=10, wcvm=9)

    def pad(s, width, align="<"):
        return f"{s:{align}{width}}"

    def fmt_p(p, det):
        return f"<{det:.3f}" if (p <= det) else f"{p:.3f}"

    L = _panel_to_lines_with_spanner(left_title, left_rows, w, fmt_p)
    R = _panel_to_lines_with_spanner(right_title, right_rows, w, fmt_p)

    gap=3
    left_w = max(len(s) for s in L)+gap
    rule = "="*(left_w + max(len(s) for s in R))
    print("\n"+rule)
    for l,r in zip_longest(L,R,fillvalue=""):
        print(f"{l:<{left_w}}{r}")
    print(rule)

def _panel_to_lines_with_spanner(title, rows, w, fmt_p):
    lines=[title]
    top = (f"{'Estimator':<20}"
           f"{'α̂':>6}"
           f"{'β̂':>7}"
           f"{'p':>10}{'D':>8}"
           f"{'p':>10}{'W':>9}")
    lines.append(top)

    det = rows[0].get("_det",0)

    for r in rows:
        lines.append(
            f"{r['Estimator']:<20}"
            f"{r['alpha']:>6.2f}"
            f"{r['beta']:>7.2f}"
            f"{fmt_p(r['KS_p'],det):>10}"
            f"{r['KS_D']:>8.3f}"
            f"{fmt_p(r['CvM_p'],det):>10}"
            f"{r['CvM_W']:>9.3f}"
        )
    return lines

In [ ]:
# ==============================================================================
# ETELN RISK MEASURES FOR DESIGN B (Different J, Same h)
# ==============================================================================


# θ = 500.0   # <- already defined earlier

# Anything above this CTE is treated as "∞" for display (just like Same-J block)
CTE_GUARD_ETELN = 2_000_000.0
_INF_THRESHOLD  = 1e15

# --------------------------------------------------------------------------
# ETELN quantile / VaR / CTE (independent of J-design)
# --------------------------------------------------------------------------

class ETELNRiskMeasuresDesignB:
    """
    Risk-measure engine for ETELN, used with Design-B L-estimators.
    The J-weights only affect (α̂, β̂); VaR/CTE use the ETELN quantile.
    """

    def __init__(self, theta=500.0, n_quad=400):
        self.theta = float(theta)
        self.n_quad = int(n_quad)

        nodes, w = leggauss(self.n_quad)
        self.u = 0.5 * (nodes + 1.0)   # [0,1] nodes
        self.w = 0.5 * w               # weights on [0,1]
        self.ln2 = np.log(2.0)

    def eteln_quantile(self, u, alpha, beta):
        """
        Quantile q(u) of ETELN(α,β,θ), consistent with your ETELN sampler:
            U0 = transform(u, β)
            ξ  = Φ^{-1}(U0)
            X  = θ * exp(ξ / α)
        """
        u = np.clip(np.asarray(u, float), 1e-12, 1.0 - 1e-12)

        if abs(beta) < 1e-10:
            u0 = np.exp(-(1.0 - u) * self.ln2)
        else:
            two_b = np.exp(beta * self.ln2)
            base  = two_b - (two_b - 1.0) * u
            base  = np.clip(base, 1e-300, None)
            u0    = base**(-1.0 / beta)

        u0 = np.clip(u0, 1e-12, 1.0 - 1e-12)
        xi  = norm.ppf(u0)
        return self.theta * np.exp(xi / alpha)

    def var(self, alpha, beta, confidence_level=0.99):
        return float(self.eteln_quantile(confidence_level, alpha, beta))

    def cte(self, alpha, beta, confidence_level=0.99):
        """
        CTE for ETELN. No analytic “α≤1” blow-up like ETELL; we just
        compute numerically and later guard extremely large values.
        """
        p = float(confidence_level)
        s = p + (1.0 - p) * self.u
        q_vals = self.eteln_quantile(s, alpha, beta)
        return float(np.sum(self.w * q_vals))

    def compute_risk_measures(self, alpha, beta,
                              confidence_levels=(0.98, 0.99, 0.995)):
        out = {}
        for cl in confidence_levels:
            var_val = self.var(alpha, beta, cl)
            cte_val = self.cte(alpha, beta, cl)

            # Numerical guard: treat *huge* CTEs as ∞ for presentation
            if np.isfinite(cte_val) and cte_val > CTE_GUARD_ETELN:
                cte_val = float('inf')

            out[cl] = {
                "confidence_level": cl,
                "tail_probability": 1.0 - cl,
                "VaR": var_val,
                "CTE": cte_val,
            }
        return out


# --------------------------------------------------------------------------
# Empirical VaR / CTE
# --------------------------------------------------------------------------

def empirical_var(x, confidence_level=0.99):
    x = np.sort(np.asarray(x, float))
    return float(np.quantile(x, confidence_level, method="linear"))

def empirical_cte(x, confidence_level=0.99):
    x = np.asarray(x, float)
    v = empirical_var(x, confidence_level)
    exc = x[x > v]
    if exc.size == 0:
        return np.nan
    return float(exc.mean())


# --------------------------------------------------------------------------
# DESIGN B risk-table computation for ETELN
# --------------------------------------------------------------------------

def compute_risk_table_designB_eteln(
    data_dict,
    estimators,
    confidence_levels=(0.98, 0.99, 0.995),
    theta=500.0,
    n_quad=400,
):
    """
    data_dict  : {"Sampled (n=50)": x_sampled, ...}
    estimators : list[(name, fit_fun)], where fit_fun(x) → (α̂, β̂)
                 (for ETELN Design-B, use make_safe_L_designB_factory_eteln)
    """
    rm = ETELNRiskMeasuresDesignB(theta=theta, n_quad=n_quad)
    results = {}

    for data_name, x in data_dict.items():
        x = np.asarray(x, float)
        x = x[x > theta]
        if x.size == 0:
            continue

        results[data_name] = {}

        # ----- Empirical measures -----
        emp_res = {}
        for cl in confidence_levels:
            v_emp = empirical_var(x, cl)
            c_emp = empirical_cte(x, cl)
            if np.isfinite(c_emp) and c_emp > CTE_GUARD_ETELN:
                c_emp = float('inf')
            emp_res[cl] = {"VaR": v_emp, "CTE": c_emp}
        results[data_name]["Empirical"] = emp_res

        # ----- Model-based measures -----
        for est_name, fit_fun in estimators:
            try:
                alpha_hat, beta_hat = fit_fun(x)
            except Exception as e:
                print(f"Warning: {est_name} failed on {data_name}: {e}")
                results[data_name][est_name] = None
                continue

            if not (np.isfinite(alpha_hat) and np.isfinite(beta_hat)):
                results[data_name][est_name] = None
                continue

            model_res = rm.compute_risk_measures(
                alpha_hat, beta_hat, confidence_levels
            )
            model_res["alpha"] = alpha_hat
            model_res["beta"]  = beta_hat
            results[data_name][est_name] = model_res

    return results


# --------------------------------------------------------------------------
# Pretty-print helpers (same style as your ETELL Design-B block)
# --------------------------------------------------------------------------

def _fmt(v):
    if v is None or (not np.isfinite(v)) or v > _INF_THRESHOLD:
        return "∞"
    return f"{v:,.2f}"

def _fmtdiff(a, b):
    bad = lambda z: (z is None) or (not np.isfinite(z)) or (z > _INF_THRESHOLD)
    if bad(a) or bad(b):
        return "—"
    return f"{(a - b):+,.2f}"


def print_risk_table_designB_eteln(
    results,
    confidence_levels=(0.98, 0.99, 0.995),
):
    for data_name, data_results in results.items():
        print("\n" + "="*100)
        print(f"ETELN Risk Measures (Design B): {data_name}")
        print("="*100)

        estimators = [k for k in data_results.keys() if k != "Empirical"]

        for cl in confidence_levels:
            tail_prob = 1.0 - cl
            print(
                f"\n{'Confidence Level':<20} "
                f"{cl*100:.1f}% (Tail Probability = {tail_prob*100:.2f}%)"
            )
            print("-"*100)

            # Header
            print(
                f"{'Method':<20} {'α̂':>8} {'β̂':>8} "
                f"{'VaR':>15} {'CTE':>15} {'VaR Diff':>15} {'CTE Diff':>15}"
            )
            print("-"*100)

            # Empirical row
            emp = data_results["Empirical"][cl]
            print(
                f"{'Empirical':<20} {'-':>8} {'-':>8} "
                f"{_fmt(emp['VaR']):>15} {_fmt(emp['CTE']):>15} "
                f"{'-':>15} {'-':>15}"
            )

            # Model rows
            for est_name in estimators:
                est_res = data_results[est_name]
                if est_res is None or cl not in est_res:
                    print(
                        f"{est_name:<20} {'FAILED':>8} {'FAILED':>8} "
                        f"{'-':>15} {'-':>15} {'-':>15} {'-':>15}"
                    )
                    continue

                alpha_hat = est_res["alpha"]
                beta_hat  = est_res["beta"]
                var_th    = est_res[cl]["VaR"]
                cte_th    = est_res[cl]["CTE"]

                print(
                    f"{est_name:<20} {alpha_hat:>8.2f} {beta_hat:>8.2f} "
                    f"{_fmt(var_th):>15} {_fmt(cte_th):>15} "
                    f"{_fmtdiff(var_th, emp['VaR']):>15} "
                    f"{_fmtdiff(cte_th, emp['CTE']):>15}"
                )


def print_comparison_table_designB_eteln(results, confidence_level=0.99):
    datasets   = list(results.keys())
    first_data = results[datasets[0]]
    estimators = [k for k in first_data.keys() if k != "Empirical"]

    print("\n" + "="*140)
    print(
        f"ETELN Risk Measures Comparison (Design B) "
        f"at {confidence_level*100:.1f}% Confidence Level"
    )
    print("="*140)

    header = f"{'Method':<20}"
    for data_name in datasets:
        header += f" | {data_name:^28}"
    print(header)

    sub = f"{'':<20}"
    for _ in datasets:
        sub += f" | {'VaR':>13} {'CTE':>13}"
    print(sub)
    print("-"*140)

    # Empirical row
    row = f"{'Empirical':<20}"
    for data_name in datasets:
        emp = results[data_name]["Empirical"][confidence_level]
        row += f" | {_fmt(emp['VaR']):>13} {_fmt(emp['CTE']):>13}"
    print(row)
    print("-"*140)

    # Model rows (MLE first)
    for est_name in ["MLE"] + [e for e in estimators if e != "MLE"]:
        row = f"{est_name:<20}"
        for data_name in datasets:
            est_res = results[data_name].get(est_name)
            if est_res is None or confidence_level not in est_res:
                row += f" | {'FAILED':>13} {'FAILED':>13}"
            else:
                var_th = est_res[confidence_level]["VaR"]
                cte_th = est_res[confidence_level]["CTE"]
                row += f" | {_fmt(var_th):>13} {_fmt(cte_th):>13}"
        print(row)

    print("="*140)


# --------------------------------------------------------------------------
# RUN BLOCK FOR ETELN DESIGN-B RISK MEASURES
# --------------------------------------------------------------------------

if __name__ == "__main__":

    # 4 datasets (same as ETELL Design B)
    data_dict_designB_eteln = {
        "Sampled (n=50)":    x_sampled,
        "Modified Sampled":  x_mod_sampled,
        "Original (n=9020)": x_original,
        "Modified Original": x_mod_original,
    }

    estimators_designB_eteln = [
    ("MLE",          lambda z, **kw: fit_mle_eteln(z, θ)),
    ("J₂(1.8,1.1)",  make_safe_L_designB_factory_eteln(11.0, 6.0, 1.8, 1.1)),
    ("J₂(1.9,1.1)",  make_safe_L_designB_factory_eteln(11.0, 6.0, 1.9, 1.1)),
    ("J₂(2.0,1.1)",  make_safe_L_designB_factory_eteln(11.0, 6.0, 2.0, 1.1)),
    ("J₂(2.2,1.1)",  make_safe_L_designB_factory_eteln(11.0, 6.0, 2.2, 1.1)),
    ]

    print("\n" + "="*100)
    print("COMPUTING ETELN RISK MEASURES FOR DESIGN B (Different J, Same h)")
    print("="*100)
    print("Datasets: 4 (Original, Modified Original, Sampled, Modified Sampled)")
    print("Design B: J₁(11.0, 6.0) fixed, varying J₂(a₂,b₂)")
    print("Confidence Levels: 98%, 99%, 99.5%")
    print("="*100)

    results_designB_eteln = compute_risk_table_designB_eteln(
        data_dict=data_dict_designB_eteln,
        estimators=estimators_designB_eteln,
        confidence_levels=(0.98, 0.99, 0.995),
        theta=θ,
        n_quad=400,
    )

    # Detailed per-dataset tables
    print_risk_table_designB_eteln(results_designB_eteln,
                                   confidence_levels=(0.98, 0.99, 0.995))

    # Side-by-side comparison tables
    print_comparison_table_designB_eteln(results_designB_eteln,
                                         confidence_level=0.98)
    print_comparison_table_designB_eteln(results_designB_eteln,
                                         confidence_level=0.99)
    print_comparison_table_designB_eteln(results_designB_eteln,
                                         confidence_level=0.995)



COMPUTING ETELN RISK MEASURES FOR DESIGN B (Different J, Same h)
Datasets: 4 (Original, Modified Original, Sampled, Modified Sampled)
Design B: J₁(11.0, 6.0) fixed, varying J₂(a₂,b₂)
Confidence Levels: 98%, 99%, 99.5%

ETELN Risk Measures (Design B): Sampled (n=50)

Confidence Level     98.0% (Tail Probability = 2.00%)
----------------------------------------------------------------------------------------------------
Method                     α̂       β̂             VaR             CTE        VaR Diff        CTE Diff
----------------------------------------------------------------------------------------------------
Empirical                   -        -        4,381.60        4,607.00               -               -
MLE                      0.95    -0.60        5,448.49        8,333.96       +1,066.89       +3,726.96
J₂(1.8,1.1)              0.57     1.35       16,684.33       40,456.30      +12,302.73      +35,849.30
J₂(1.9,1.1)              0.57     1.17       17,395.71       41,